In [1]:
import os
import sys
import time
import numpy as np
import imgaug 

In [2]:
from mrcnn.config import Config
from mrcnn import model as modellib, utils

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils
from coco.coco import CocoDataset

Using TensorFlow backend.


In [3]:
COCO_PRETRAINED_WEIGHTS_PATH = os.path.abspath("mask_rcnn_coco.h5")
DATASET_PATH = os.path.abspath("DATASETS/")
TRAIN_ANNOTATIONS_PATH = os.path.abspath("DATASETS/annotations/instances_train2017.json")
LOGS_PATH = os.path.abspath("LOGS")

In [ ]:
class CocoConfig(Config):
    """Configuration for training on MS COCO.
    Derives from the base Config class and overrides values specific
    to the COCO dataset.
    """
    # Give the configuration a recognizable name
    NAME = "coco"
    
    IMAGES_PER_GPU = 2 
    #the model fits only once on my 6GB GTX1060. 
    #fits twice on 12GB Tesla P100
    
    NUM_CLASSES = 1 + 80  #80 COCO classes + BG class

In [ ]:
coco_train_annotatons = COCO(TRAIN_ANNOTATIONS_PATH)

loading annotations into memory...
Done (t=14.11s)
creating index...
index created!


In [ ]:
#Train dataset
dataset_train = CocoDataset()
dataset_train.load_coco(dataset_dir=DATASET_PATH, subset="train", year=2017, class_ids=None,
                        class_map=None, return_coco=False, auto_download=False)
dataset_train.prepare()



# Validation dataset
dataset_val = CocoDataset()
dataset_val.load_coco(dataset_dir=DATASET_PATH, subset="val", year=2017, auto_download=False)
dataset_val.prepare()

loading annotations into memory...
Done (t=14.58s)
creating index...
index created!
loading annotations into memory...
Done (t=0.35s)
creating index...
index created!


In [ ]:
import numpy as np

augmentation = imgaug.augmenters.Fliplr(0.5)


histories = []

for i in range(10):
    
    config = CocoConfig()

    config.LEARNING_RATE = np.random.uniform(0.01, 0.0001)
    config.LEARNING_MOMENTUM = 0.9

    # Weight decay regularization
    config.WEIGHT_DECAY = np.random.uniform(0.001, 0.00001)
    
    print("TRAINING #{}, LR: {}, DECAY: {}".format(i, config.LEARNING_RATE, config.WEIGHT_DECAY))

    # Create model
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=LOGS_PATH)
        
    # Training - Stage 1
    print("Training network heads")
    hist1 = model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=1,
                layers='heads',
                augmentation=augmentation)

    # Training - Stage 2
    # Finetune layers from ResNet stage 4 and up
    print("Fine tune Resnet stage 4 and up")
    hist2 = model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=5,
                layers='4+',
                augmentation=augmentation)

    # Training - Stage 3
    # Fine tune all layers
    print("Fine tune all layers")
    hist3 = model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE / 5,
                epochs=2,
                layers='all',
                augmentation=augmentation)
    
    histories.append({'model': model, 'hist1': hist1, 'hist2': hist2, 'hist3': hist3, 'lr': config.LEARNING_RATE, 'decay': config.WEIGHT_DECAY})

TRAINING #0, LR: 0.0006762299818903143, DECAY: 0.0005320091691430175
Training network heads

Starting at epoch 0. LR=0.0006762299818903143

Checkpoint Path: /home/013855803/CMPE-297-homework/LOGS/coco20190407T0734/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2

/home/013855803/anaconda3/envs/mrcnn/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/013855803/anaconda3/envs/mrcnn/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/1
1000/1000 [==============================] - 1437s 1s/step - loss: 4.4954 - rpn_class_loss: 0.4504 - rpn_bbox_loss: 1.9779 - mrcnn_class_loss: 0.3303 - mrcnn_bbox_loss: 1.0510 - mrcnn_mask_loss: 0.6857 - val_loss: 12.3657 - val_rpn_class_loss: 4.7941 - val_rpn_bbox_loss: 4.8580 - val_mrcnn_class_loss: 0.5084 - val_mrcnn_bbox_loss: 1.4780 - val_mrcnn_mask_loss: 0.7271
Fine tune Resnet stage 4 and up

Starting at epoch 1. LR=0.0006762299818903143

Checkpoint Path: /home/013855803/CMPE-297-homework/LOGS/coco20190407T0734/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b         (Conv2D)
bn4a_branch2b          (BatchNorm)
res4a_branch2c         (Conv2D)
res4a_branch1          (Conv2D)
bn4a_branch2c          (BatchNorm)
bn4a_branch1           (BatchNorm)
res4b_branch2a         (Conv2D)
bn4b_branch2a          (BatchNorm)
res4b_branch2b         (Conv2D)
bn4b_branch2b          (BatchNorm)
res4b_br


TRAINING #1, LR: 0.00829352558545438, DECAY: 0.00033657542559360916
Training network heads

Starting at epoch 0. LR=0.00829352558545438

Checkpoint Path: /home/013855803/CMPE-297-homework/LOGS/coco20190407T1041/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2  

Epoch 2/8
 280/1000 [=======>......................] - ETA: 26:46 - loss: 3.0757 - rpn_class_loss: 0.4396 - rpn_bbox_loss: 1.2453 - mrcnn_class_loss: 0.2407 - mrcnn_bbox_loss: 0.6282 - mrcnn_mask_loss: 0.5218

In [ ]:
model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=args.logs)

# model = modellib.MaskRCNN(mode="inference", config=config,
#                           model_dir=args.logs)